In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import plotly.graph_objects as go
import networkx as nx

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import image
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf
from mtcnn import MTCNN


from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model, load_model
import cv2
import os

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader, Loader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from spektral.utils import normalized_laplacian

import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(frac=0.01,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,gender
0,447,"Oh! Oh, I think I'm gonna throw up a little bi...",Rachel,disgust,negative,40,2,8,5,"00:06:45,822","00:06:50,575",female
1,557,"Sure, New Year's Eve is only two weeks away. C...",Janine,neutral,neutral,53,3,6,10,"00:20:02,451","00:20:06,996",female
2,290,"But, you've got one more apology to make, all...",Joey,neutral,neutral,24,3,3,11,"00:19:55,194","00:20:00,740",male
3,327,"Okay, okay. It's James Brolin. James Brolin is...",Phoebe,neutral,neutral,27,7,8,1,"00:01:44,270","00:01:51,819",female
4,662,Did you just make that up?,Rachel,surprise,positive,63,12,8,18,"0:20:16,959","0:20:18,759",female
5,241,And it's not even born yet.,Phoebe,sadness,negative,19,20,1,23,"00:15:25,424","00:15:27,842",female
6,154,Hey Toby!,Bob,joy,positive,14,0,8,5,"0:16:07,675","0:16:08,267",female
7,840,God. I forgot how much I love driving. I have,Rachel,joy,positive,78,0,7,22,"00:09:32,530","00:09:35,490",female
8,419,"Okay, I do.",Rachel,neutral,neutral,36,1,3,21,"0:15:30,512","0:15:31,324",female
9,833,It's just if-if we got together again and it d...,Ross,sadness,negative,77,13,8,24,"00:18:07,658","00:18:16,624",male


In [3]:
# Load MTCNN model for face detection
detector = MTCNN()

# Load FaceNet model for face embedding extraction
facenet_model = tf.keras.applications.ResNet50(weights='imagenet')

def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0 
    return img

def extract_face_embeddings(frame):
    faces = detector.detect_faces(frame)
    face_embeddings = []
    for face in faces:
        x, y, w, h = face['box']
        x1, y1 = max(x, 0), max(y, 0)
        x2, y2 = min(x + w, frame.shape[1]), min(y + h, frame.shape[0])
        cropped_face = frame[y1:y2, x1:x2]
        
        # Preprocess the face for FaceNet model
        preprocessed_face = preprocess_image(cropped_face)
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Get face embeddings using FaceNet model
        face_embedding = facenet_model.predict(preprocessed_face)
        face_embeddings.append(face_embedding)
    return face_embeddings

# Read video
folder_path = 'MELD.Raw/dev_splits_complete/'
one_face_videos = {}

for idx,row in df.iterrows():
    file_name = 'dia' + str(row['Dialogue_ID']) + '_utt' + str(row['Utterance_ID']) + '.mp4'
    video_path = folder_path + file_name
    if os.path.isfile(video_path):
        video_capture = cv2.VideoCapture(video_path)  # Provide the path to your input video

        single_video_embeddings = []  # List to store embeddings for all frames

        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            #check for more than 1 faces
            if len(detector.detect_faces(frame)) == 1:
            
                # Extract face embeddings from each frame
                extracted_embeddings = extract_face_embeddings(frame)
                
                single_video_embeddings.append(extracted_embeddings)  # Append embeddings for this frame
                
                # Display the frame with bounding boxes around detected faces
                for face in detector.detect_faces(frame):
                    x, y, w, h = face['box']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                cv2.imshow('Video', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                video_capture.release()
                cv2.destroyAllWindows()

            else:
                break
        if single_video_embeddings != []:            
            one_face_videos[file_name] = single_video_embeddings
        else:
            one_face_videos[file_name] = 'too many faces'

                




1/1 [==============================] - 0s 24ms/step


In [4]:
df['faces'] = one_face_videos.values()

df = df[df.faces != 'too many faces']

df['faces'] = df.faces.apply(lambda x: x[0][0][0])


C:\Users\gsevr\AppData\Local\Temp\ipykernel_4356\1239130037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['faces'] = df.faces.apply(lambda x: x[0][0][0])


In [5]:
padding_length = len(df) + len(df.Emotion.unique()) + len(df.gender.unique())

def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]


texts = df.prepro.values
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=padding_length, padding='post')

C:\Users\gsevr\AppData\Local\Temp\ipykernel_4356\790835697.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])


In [6]:
glove_path = 'glove.6B.50d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, encoding='utf8',no_header=True)



In [7]:
embedding_dim = 50  
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

embedding_layer = Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=padding_length,  #for matmul
    trainable=False
)


model = Sequential()
model.add(embedding_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics='accuracy')


In [8]:
embedding_vectors = model.predict(padded_sequences)
df['text'] = list(embedding_vectors)

1/1 [==============================] - 0s 35ms/step


C:\Users\gsevr\AppData\Local\Temp\ipykernel_4356\1340693711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = list(embedding_vectors)


In [9]:
G = nx.Graph()


In [10]:

emotion_col = {
            'sadness':'teal',
            'surprise':'silver',
            'neutral':'lightgreen',
            'joy':'purple',
            'anger':'black',
            'disgust': 'brown',
            'fear':'yellow'
}

gender_col = {
    'male':'blue',
    'female':'pink'
}

In [11]:
for emotion in df.Emotion.unique():
    G.add_node(emotion, color=emotion_col[emotion],hover=emotion,emotion=emotion, type='emotion')


In [12]:
for gender in df.gender.unique():
    G.add_node(gender,color=gender_col[gender], hover=gender, gender=gender, type='gender')

In [13]:
for idx, row in df.iterrows():
    G.add_node(row['Utterance'], 
    text_embedding=row['text'],
    visual_embedding=row['faces'],
    color=emotion_col[row['Emotion']],
    hover=row['Utterance'],
    emotion=row['Emotion'],
    gender=row['gender'], 
    type='utterance',
    padding='zero')

In [14]:
for idx, row in df.iterrows():
    G.add_edge(row['Utterance'], row['Emotion'],color=emotion_col[row['Emotion']],weight=1)
    G.add_edge(row['Utterance'], row['gender'], color=gender_col[row['gender']],weight=1)


In [15]:
pos = nx.spring_layout(G)
# pos = nx.fruchterman_reingold_layout(G)


In [16]:
import textwrap

fig = go.Figure()

node_positions = {}

for node, position in pos.items():
    x, y = position  

    color = G.nodes[node]['color']  
    hover = G.nodes[node]['hover']
    hover = '<br>'.join(textwrap.wrap(hover,width=50))
    try:
        emotion = G.nodes[node]['emotion']
    except KeyError:
        try:
            gender = G.nodes[node]['gender']
        except KeyError:
            continue
    node_type = G.nodes[node]['type']

    if node_type == 'utterance':
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=emotion, hovertemplate=hover))
    else:
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=node_type, hovertemplate=hover))


for edge in G.edges():
    color = G.edges[edge]['color']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(width=1,color=color), name='Edge'))

fig.update_layout(showlegend=False, title='Emotions Graph')

fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig.show()

In [17]:
# Adj matrix

adjacency_info = dict(G.adj)

nodes = list(G.nodes())

adj_matrix = np.zeros((len(nodes), len(nodes)))

for i, node in enumerate(nodes):
    for j, other_node in enumerate(nodes):
        if other_node in adjacency_info[node]:
            adj_matrix[i, j] = 1 

print(adj_matrix)


[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [18]:
# try spektral

adjacency_matrix = nx.adjacency_matrix(G)  # Adjacency matrix
features_text = np.array([G.nodes[node]['text_embedding'] for node in G.nodes() if G.nodes[node]['type'] == 'utterance']).T  # 'text' features, perm
features_video = np.array([G.nodes[node]['visual_embedding'] for node in G.nodes() if G.nodes[node]['type'] == 'utterance'])  # 'video' features

adj_normalized = normalized_laplacian(adjacency_matrix)



In [19]:
y_train = [G.nodes[node]['emotion'] for node in G.nodes if G.nodes[node]['type'] == 'emotion']

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

y_train = ohe.fit_transform(np.array(y_train).reshape(-1,1))

y_train = tf.constant(y_train.toarray())

In [20]:
# features_text = tf.constant(np.pad(features_text,((0,5))))
# features_video = tf.constant(np.pad(features_video,((0,5))))
# y_train = tf.constant(np.pad(y_train,((0,9))))

In [21]:
from tensorflow.keras.layers import Input, Dropout, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [22]:
text_in = Input(shape=(features_text.shape[1],),name='text_input_layer')
video_in = Input(shape=(features_video.shape[1]),name='video_input_layer')
adj_in = Input((adj_matrix.shape[0],), sparse=True, name='adj_matrix')

In [23]:
from tensorflow.keras.layers import Permute

gcn1 = GCNConv(16, activation='relu')([text_in, adj_in])
gcn1 = Dropout(0.5)(gcn1)
gcn2 = GCNConv(8, activation='relu')([gcn1, adj_in])


In [24]:
merged_features = Concatenate()([gcn2, video_in])

In [25]:
fc = Dense(16, activation='relu')(merged_features)
output = Dense(len(df.Emotion.unique()), activation='softmax')(fc)

In [26]:
model = Model(inputs=[text_in, video_in, adj_in], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
print(features_text.shape)
print(features_video.shape)
print(y_train.shape)
print(adj_normalized.shape)

(50, 13, 6)
(6, 1000)
(5, 5)
(13, 13)


In [55]:
transpose_text = tf.transpose(features_text,perm=[2,0,1])
print(transpose_text.shape)
transpose_video = tf.reshape(features_video,(50,20,features_video.shape[0]))
transpose_video = tf.transpose(transpose_video,perm=[2,0,1])
print(transpose_video.shape)



(6, 50, 13)
(6, 50, 20)


In [56]:
history = model.fit([transpose_text, transpose_video, tf.constant(adj_normalized.toarray())],
          y_train,
          epochs=100,
          batch_size=32,
          )

ValueError: Data cardinality is ambiguous:
  x sizes: 6, 6, 13
  y sizes: 5
Make sure all arrays contain the same number of samples.